In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader,random_split

In [3]:
df = pd.read_csv('train.csv');
df.head()

,sms,label
0,Ok i am on the way to home hi hi\n,0
1,Ur balance is now £500. Ur next question is: W...,1
2,I'm tired of arguing with you about this week ...,0
3,Tell them the drug dealer's getting impatient\n,0
4,Hmmm... Guess we can go 4 kb n power yoga... H...,0


In [4]:
texts = df['sms'].tolist();
Y = df['label'].tolist();

In [5]:
texts[100]

'Feel Yourself That You Are Always Happy.. Slowly It Becomes Your Habit &amp; Finally It Becomes Part Of Your Life.. Follow It.. Happy Morning &amp; Have A Happy Day:)\n'

In [6]:
import re
def f(t):
  t = re.sub(r'\n','',t);
  t = re.sub(r"[^a-zA-Z\s\'&]"," ",t).lower();
  t = re.sub(r'\s+',' ',t);
  return t

In [7]:
f(texts[2000]),texts[2000]

("k i did't see you k where are you now ",
 "K.i did't see you.:)k:)where are you now?\n")

In [8]:
texts = [f(t) for t in texts];

In [19]:
from collections import Counter
class Vocab:
  def __init__(self, texts,stopwords = set(),min_freq=10,min_len=3):
    self.vocab = {'<pad>','<unk>'};
    self.word2idx = {"<pad>":0,'<unk>':1};
    self.idx2word = {0:"<pad>",1:"<unk>"};
    index = 2;
    sentences = [t.split() for t in texts];
    cnt = Counter(word for sentence in sentences for word in sentence);
    sentences = [[word for word in sentence if word not in stopwords and len(word)>=min_len and cnt[word]>=min_freq]
                 for sentence in sentences];
    for sentence in sentences  :
      for word in sentence :
        if (word not in self.vocab):
          self.vocab.add(word);
          self.word2idx[word] = index;
          self.idx2word[index] = word;
          index+=1;
  def encode(self, text):
    return torch.tensor([self.word2idx.get(t,1) for t in text.split()],dtype=torch.int64);
  def decode(self, ids):
    return ' '.join([self.idx2word[t.item()] for t in ids]);


In [20]:
vocab = Vocab(texts,min_freq=0);

In [21]:
len(vocab.vocab)

6784

In [22]:
class Text(Dataset):
  def __init__(self, texts,labels):
    super().__init__();
    self.texts = texts;
    self.labels = labels;
  def __len__(self): return len(self.texts);
  def __getitem__(self, index):
    text = vocab.encode(self.texts[index]);
    label = self.labels[index];
    return text,label;

In [13]:
dataset = Text(texts,Y);
dataset[10]

([72, 73, 74, 75, 76, 77, 78], 0)

In [23]:
dataset[210]

(tensor([   1,   48,  219,  303,    1,   21,  181,  269, 1137,    1, 1138]), 0)

In [24]:
len(dataset)

4459

In [25]:
def padding(ids,max_length):
  ids = ids.tolist();
  ids = ids + [0] * (max_length - len(ids));
  return ids;
def collate_fn(batch):
  x,y = zip(*batch);
  mx = max(len(xi) for xi in x);
  x = torch.tensor([padding(xi,mx) for xi in x],dtype = torch.int64);
  y = torch.tensor(y, dtype = torch.int64);
  return x, y;

In [26]:
train_dataset,val_dataset = random_split(dataset,[4000,459])
train_loader = DataLoader(train_dataset,batch_size=128,shuffle=True,collate_fn=collate_fn);
val_loader = DataLoader(val_dataset,batch_size=128,shuffle=False,collate_fn=collate_fn);


In [27]:
for x, y in train_loader :
  print(x,y);
  break

tensor([[2728,  282,    1,  ...,    0,    0,    0],
        [  30,  442,    1,  ...,    0,    0,    0],
        [1419,  315,  163,  ...,    0,    0,    0],
        ...,
        [ 801,    1,  685,  ...,    0,    0,    0],
        [ 236,    1,    1,  ...,    0,    0,    0],
        [ 388, 3035, 6597,  ...,    0,    0,    0]]) tensor([1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


In [30]:
class RNN(nn.Module):
  def __init__(self, vocab_size,embedding_size,hidden_size):
    super().__init__();
    self.embedding = nn.Embedding(vocab_size,embedding_size,padding_idx = 0)
    self.rnn = nn.LSTM(embedding_size,hidden_size//2,num_layers=4,bidirectional=True,
                       batch_first = True);
    self.fc = nn.Linear(hidden_size,2);
  def forward(self, x):
    embed = self.embedding(x);
    _,(h,_) = self.rnn(embed);
    h = torch.cat([h[-2],h[-1]],dim = 1);
    return self.fc(h);

In [40]:
model = RNN(vocab_size = len(vocab.word2idx),embedding_size=200,hidden_size=512);
device = 'cuda';
model.to(device);

In [41]:
optimizer = torch.optim.Adam(lr = 2e-3,params = model.parameters());
loss_function = nn.CrossEntropyLoss()

In [42]:
b = 0;
for i in range(10):
  model.train();
  for x, y in train_loader :
    x,y = x.to(device),y.to(device)
    optimizer.zero_grad();
    out = model(x);
    loss = loss_function(out,y);
    loss.backward();
    optimizer.step();
    b+=1;
    if (b%100==0): print(round(loss.item(),4));

0.0415
0.004
0.0009


In [43]:
model.eval();
predicts = []
y_test = []
with torch.no_grad():
  for x, y in val_loader :
    x, y = x.to(device),y.to(device);
    predict = model(x);
    predict = predict.argmax(1);
    predicts.extend(predict.tolist());
    y_test.extend(y.tolist())

In [44]:
from sklearn.metrics import accuracy_score,f1_score

In [45]:
print(accuracy_score(y_test,predicts));
print(f1_score(y_test,predicts))

0.9825708061002179
0.9259259259259259


In [ ]:
##### Transformers #######

In [47]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

In [48]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name);

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [49]:
max(len(t.split()) for t in texts)

189

In [50]:
class Text2(Dataset):
  def __init__(self, texts,labels):
    super().__init__();
    self.texts = texts;
    self.labels = labels;
  def __len__(self): return len(self.texts);
  def __getitem__(self, index):
    text = self.texts[index];
    label = self.labels[index];
    tokens = tokenizer(text,padding='max_length',max_length=256,return_tensors='pt',
                       truncation=True);
    item = {key:val.squeeze(0) for key,val in tokens.items()};
    item['labels'] = torch.tensor(label,dtype = torch.int64);
    return item

In [51]:
dataset = Text2(texts,Y);
dataset[0]

{'input_ids': tensor([ 101, 7929, 1045, 2572, 2006, 1996, 2126, 2000, 2188, 7632, 7632,  102,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, 

In [57]:
train_dataset,val_dataset = random_split(dataset,[4000,459])
train_loader = DataLoader(train_dataset,batch_size=16,shuffle=True);
val_loader = DataLoader(val_dataset,batch_size=16,shuffle=False)

In [54]:
model2 = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
model2.to(device);

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
optimizer = torch.optim.Adam(lr=2e-5,params = model2.parameters());

In [58]:
b = 0;
for i in range(2):
  model2.train();
  for x in train_loader :
    attention_mask = x['attention_mask'].to(device);
    input_ids = x['input_ids'].to(device);
    labels = x['labels'].to(device);
    optimizer.zero_grad();
    out = model2(attention_mask = attention_mask,input_ids=input_ids,labels=labels);
    loss = out.loss;
    loss.backward();
    optimizer.step();
    b+=1;
    if (b%10==0): print(round(loss.item(),4));

0.3712
0.1605
0.3257
0.0367
0.048
0.0528
0.1918
0.0364
0.0659
0.2119
0.2494
0.0116
0.0098
0.0144
0.0138
0.0248
0.3019
0.0379
0.0071
0.0244
0.113
0.0135
0.0225
0.0347
0.04
0.0072
0.0044
0.0048
0.0829
0.003
0.0176
0.0028
0.0146
0.0021
0.0078
0.0124
0.0021
0.0045
0.0336
0.0018
0.0022
0.0079
0.0023
0.0038
0.0068
0.0012
0.3188
0.0569
0.006
0.0778


In [61]:
model2.eval();
predicts = []
y_test = []
with torch.no_grad():
  for x in val_loader :
    attention_mask = x['attention_mask'].to(device);
    input_ids = x['input_ids'].to(device);
    labels = x['labels'].to(device);
    predict = model2(attention_mask=attention_mask,input_ids=input_ids,labels=labels);
    predict = predict.logits.argmax(1);
    predicts.extend(predict.tolist());
    y_test.extend(labels.tolist())

In [62]:
print(accuracy_score(y_test,predicts));
print(f1_score(y_test,predicts))

0.9891067538126361
0.9572649572649573


In [63]:
test = pd.read_csv('test.csv');
test.head()

,id,sms
0,0,No message..no responce..what happend?\n
1,1,At WHAT TIME should i come tomorrow\n
2,2,Come to my home for one last time i wont do an...
3,3,Get ur 1st RINGTONE FREE NOW! Reply to this ms...
4,4,See you there! \n


In [65]:
test_text = test['sms'].tolist()
Id = test['id'].to_numpy();

In [66]:
test_texts = [f(t) for t in test_text];


In [67]:
test_dataset = Text2(test_texts,[0] * len(test_texts));
test_loader = DataLoader(test_dataset,batch_size=16,shuffle=False);

In [68]:
model2.eval();
predicts = []
with torch.no_grad():
  for x in test_loader :
    attention_mask = x['attention_mask'].to(device);
    input_ids = x['input_ids'].to(device);
    labels = x['labels'].to(device);
    predict = model2(attention_mask=attention_mask,input_ids=input_ids,labels=labels);
    predict = predict.logits.argmax(1);
    predicts.extend(predict.tolist());


In [70]:
len(predicts),len(test_texts)

(1115, 1115)

In [71]:
predicts = np.array(predicts)

In [72]:
df.head()

,sms,label
0,Ok i am on the way to home hi hi\n,0
1,Ur balance is now £500. Ur next question is: W...,1
2,I'm tired of arguing with you about this week ...,0
3,Tell them the drug dealer's getting impatient\n,0
4,Hmmm... Guess we can go 4 kb n power yoga... H...,0


In [73]:
sub = pd.DataFrame({'id':Id,'label':predicts});
sub.head()

,id,label
0,0,0
1,1,0
2,2,0
3,3,1
4,4,0


In [74]:
sub.to_csv('sub.csv',index=False);